In [8]:
import warnings
warnings.filterwarnings('ignore')
import os
import tensorflow as tf


import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from urllib.request import urlopen,urlretrieve

from tensorflow.keras.models import load_model
from sklearn.datasets import load_files   
#from tensorflow.keras.utils import np_utils
from glob import glob
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras.callbacks import TensorBoard, ReduceLROnPlateau, ModelCheckpoint

class_names = sorted(os.listdir(r"/home/nvme/data/train/train"))

N_classes = len(class_names)

In [9]:
base_model = tf.keras.applications.inception_v3.InceptionV3(
    weights = "imagenet",
    input_shape = (224,224,3),
    include_top = False)

In [10]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(4096, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(N_classes, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [11]:
from tensorflow.keras.optimizers import SGD, Adam

# compile the model (should be done *after* setting layers to non-trainable)
adam = Adam(lr = 0.0001)

model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
# example of progressively loading images from file
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# create generator
datagen = ImageDataGenerator(
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    #rotation_range=20,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    rescale = 1./255.,
    #horizontal_flip=True,
    validation_split=0.2)

train_gen = datagen.flow_from_directory('/home/nvme/data/train/train',
                                        class_mode = "categorical",
                                        target_size = (224, 224),
                                        batch_size = 32,
                                        shuffle = True,
                                        subset = "training",
                                        seed = 42)
val_gen = datagen.flow_from_directory('/home/nvme/data/train/train',
                                        class_mode = "categorical",
                                        target_size = (224, 224),
                                        batch_size = 32,
                                        shuffle = True,
                                        subset = "validation",
                                        seed = 42)

Found 22443 images belonging to 17 classes.
Found 5602 images belonging to 17 classes.


In [ ]:
epochs = 10

model.fit_generator(train_gen,
    steps_per_epoch = train_gen.samples // 32,
    validation_data = val_gen, 
    validation_steps = val_gen.samples // 32,
    epochs = epochs 
    #,callbacks=[tensorboard_callback]
                   )

Epoch 1/10
701/701 [==============================] - 858s 1s/step - loss: 0.7906 - accuracy: 0.7608 - val_loss: 0.7074 - val_accuracy: 0.8141
Epoch 2/10
701/701 [==============================] - 861s 1s/step - loss: 0.6019 - accuracy: 0.8049 - val_loss: 0.7070 - val_accuracy: 0.8227
Epoch 3/10
701/701 [==============================] - 872s 1s/step - loss: 0.5519 - accuracy: 0.8219 - val_loss: 0.6185 - val_accuracy: 0.8411
Epoch 4/10
701/701 [==============================] - 864s 1s/step - loss: 0.5185 - accuracy: 0.8295 - val_loss: 0.7316 - val_accuracy: 0.8066
Epoch 5/10
701/701 [==============================] - 865s 1s/step - loss: 0.4944 - accuracy: 0.8350 - val_loss: 0.7293 - val_accuracy: 0.8011
Epoch 6/10
701/701 [==============================] - 872s 1s/step - loss: 0.4796 - accuracy: 0.8429 - val_loss: 0.6553 - val_accuracy: 0.8398
Epoch 7/10
701/701 [==============================] - 858s 1s/step - loss: 0.4555 - accuracy: 0.8442 - val_loss: 0.6929 - val_accuracy: 0.8316

In [ ]:
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True
    
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.00001, momentum=0.9),
                            loss='categorical_crossentropy',
                            metrics=['accuracy'])

model.fit_generator(train_gen,
    steps_per_epoch = train_gen.samples // 32,
    validation_data = val_gen, 
    validation_steps = val_gen.samples // 32,
    epochs = epochs)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen_test = ImageDataGenerator(rescale = 1./255.)

test_gen = datagen_test.flow_from_directory('/home/nvme/data/test',
                                        #class_mode = "categorical",
                                        target_size = (224, 224),
                                        batch_size = 1,
                                        shuffle = False)

pred = model.predict_generator(test_gen, verbose = 1)

In [ ]:
import numpy as np
p = np.argmax(pred, axis = 1)
predictions = [class_names[k] for k in p]
a = np.arange(len(predictions))
d = {'Id': a, 'Category': predictions}

import pandas as pd
df = pd.DataFrame(d)

df.to_csv("submission2.csv", index = None, header = True)